# Active Directory Module Notebook

This Jupyter Notebook explores the various Active Directory Module Cmdlets and sample usage. This notebook is intended to be run on systems that can add the `RSAT-AD-PowerShell` Windows Feature. All the Active Directory CmdLets used in this notebook are read-only (i.e., Get-). No data will be written to Active Directory and this notebook should not be run by a member of a Privileged Group. These are simplistic examples and will not work with some inter-domain/inter-forest lookups. Those queries would make an excellent update.

In [1]:
$RunTime = [System.Diagnostics.Stopwatch]::StartNew()

## Active Directory Module Installation and Loading

### Install Active Directory PowerShell Module

If the Active Directory module is not installed on your system, this will install the Windows Feature `RSAT-AD-PowerShell` on it.

In [2]:
$FeatureName = 'RSAT-AD-PowerShell'
$FeatureFound = Get-WindowsFeature -Name $FeatureName | 
    Select-Object -ExpandProperty 'Installed'
if($FeatureFound -eq $false) {
    Install-WindowsFeature -Name $FeatureName -Verbose
} else {
    Write-Output "$FeatureName is already installed"
}

### Trust but Verify

In [3]:
$ModuleTest = Get-Module -Name 'ActiveDirectory' -ListAvailable
if($ModuleTest) { 
    Write-Output 'The Active Directory module is installed.'
    $ModuleTest | 
        Format-Table -Property @(
            'Name',
            'Version',
            @{Name='Total CmdLets';Expression={$_.ExportedCmdlets.Count}},
            @{Name='Total Get CmdLets';Expression={($_.ExportedCmdlets.Keys | Where-Object { $_ -match 'Get-' }).Count}}
        )
} else {
    Write-Output "I don't think you are going to get too far with this notebook if the Active Directory module is not present on your computer."
}

### Import Active Directory Module

This will force the module to be loaded.

In [4]:
Import-Module -Name 'ActiveDirectory' -Force

## Do I like to run code as a Tier 0 Account?

In [5]:
$Found = $null
$Username = $env:USERNAME
$MyDistinguishedName = Get-ADUser -Identity $Username |
    Select-Object -ExpandProperty 'DistinguishedName'
if($MyDistinguishedName) {
    $AdministratorList = Get-ADGroupMember -Identity 'Administrators' -Recursive |
        Select-Object -ExpandProperty 'DistinguishedName'
    if($AdministratorList) {
        $Found = $AdministratorList |
            Where-Object { $_ -contains $MyDistinguishedName }
    }
}
if($Found) {
    Write-Output "*** $Username likes to run code off the Internet with a Tier 0 account! ***"
} else {
    Write-Output "$Username does not run code off the Internet with a Tier 0 account."
}

## Domain Controllers

### Find A Domain Controller

Make sure you run this code section before running ones after this one. Most are dependent on the `$DomainController` object.

In [6]:
$DomainController = Get-ADDomainController -Service 'ADWS' -Discover
$DomainController.HostName[0]

### List of all Domain Controllers

In [7]:
$DomainControlerCollection = Get-ADDomainController -Filter *
$DomainControlerCollection |
    Format-Table -Property @('HostName','Site','OperatingSystem','ComputerObjectDN')

## Forest Information

### Get Forest Information

In [8]:
$ForestInformation = Get-ADForest -Server $DomainController
$ForestInformation

### RootDSE

In [9]:
Get-ADRootDSE -Server $DomainController

### Forest Mode

In [10]:
$ForestInformation.ForestMode

### Forest FSMO Role Holders

In [11]:
$ForestInformation |
    Select-Object -Property @(
        'DomainNamingMaster'
        'SchemaMaster'
    ) |
    Format-List

### Active Directory Optional Features

In [12]:
Get-ADOptionalFeature -Filter * -Server $DomainController |
    Format-Table -Property @('Name','EnabledScopes')

### Export Sites & Subnets

In [13]:
$SiteCollection = Get-ADReplicationSite -Filter * -Server $DomainController
$SubnetPropertyList = @('CanonicalName', 'CN', 'createTimeStamp', 'Description', 'DisplayName', 'DistinguishedName', 'modifyTimeStamp', 'Name', 'ObjectGuid', 'ProtectedFromAccidentalDeletion')
[System.Array]$SubnetCollection = foreach($Site in $SiteCollection) {
  $SiteDistinguishedName = $Site.DistinguishedName
  $SubnetCollection = Get-ADReplicationSubnet -Filter { Site -eq $SiteDistinguishedName } -Properties $SubnetPropertyList -Server $DomainController
  if($SubnetCollection) {
    foreach($SiteSubnet in $SubnetCollection) {
      $SubnetObject = [PSCustomObject]@{
        Site            = $Site.Name
        SiteDescription = $Site.Description
        SubnetName      = $SiteSubnet.Name
        Network         = $SiteSubnet.Name.Split('/')[0]
        Prefix          = $SiteSubnet.Name.Split('/')[1]
        Description     = $SiteSubnet.Description
        Created         = $SiteSubnet.createTimeStamp.ToString()
        Modified        = $SiteSubnet.modifyTimeStamp.ToString()
      }
      $SubnetObject
    }
  }
}
$SubnetCollection |
  Format-Table


## Domain Information

### Get Domain Information

In [14]:
$DomainInformation = Get-ADDomain -Server $DomainController
$DomainInformation

### Domain Mode

In [15]:
$DomainInformation.DomainMode

### Domain FSMO Role Holders

In [16]:
$DomainInformation |
    Select-Object -Property @(    
        'PDCEmulator'
        'InfrastructureMaster'
        'RIDMaster'
    ) |
    Format-List

### Domain Trusts

In [17]:
Get-ADTrust -Filter * -Server $DomainController

### Treat the Domain like a File System

In [18]:
Import-Module -Name 'ActiveDirectory' -Force
Get-PSDrive |
    Where-Object -Property 'Provider' -Match 'ActiveDirectory' |
    Format-List
Write-Output "Root of the Domain $($DomainInformation.DNSRoot)"
Get-ChildItem -Path "AD:\$($DomainInformation.DistinguishedName)"
Get-Item -Path "AD:\$($DomainInformation.DistinguishedName)"

## Privileged Accounts & Groups (Tier 0)

### Sneaky Domain Admin?

In [19]:
$DomainAdminSID = "$($DomainInformation.DomainSID.AccountDomainSid.Value)-500"
$DomainAdministrator = Get-ADUser -Filter { SID -like $DomainAdminSID} -Server $DomainController
if($DomainAdministrator.Name -ne 'Administrator') {
    Write-Output "$($DomainInformation.NetBiosName) has renamed the Administrator account to $($DomainAdministrator.Name). I almost didn't notice. Almost."
} else {
    Write-Output "No security through obscurity for you in the $($DomainInformation.DNSRoot) domain."
}

### Members of all Privileged Groups

In [20]:
$PrivilegedGroupList = @(
    'Account Operators'
    'Administrators'
    'Backup Operators'
    'Cert Publishers'
    'Cryptographic Operators'
    'DHCP Administrators'
    'DnsAdmins'
    'Domain Admins'
    'Enterprise Admins'
    'Group Policy Creator Owners'
    'Print Operators'
    'Remote Desktop Services Users'
    'Schema Admins'
    'Server Operators'
)
$MemberCollection = foreach($PrivilegedGroup in $PrivilegedGroupList) {
    $GroupObject = Get-ADGroup -Filter  { Name -eq $PrivilegedGroup }
    if($GroupObject) {
        Get-ADGroupMember -Identity $PrivilegedGroup -Server $DomainController -Recursive
    }
}
$MemberCollection |
    Sort-Object -Property 'DistinguishedName' -Unique |
    Format-Table -Property @('SamAccountName','DistinguishedName')

### List the DistinguishedName of all users recursively in the Administrators Group

In [21]:
Get-ADGroupMember -Identity 'Administrators' -Server $DomainController -Recursive |
    Select-Object -ExpandProperty 'distinguishedName'

### Is the Schema Admins Empty?

In [22]:
$SchemaAdminsCollection = Get-ADGroupMember -Identity 'Schema Admins' -Server $DomainController
if($SchemaAdminsCollection) {
    Write-Output "The 'Schema Admins' group in $($DomainInformation.DNSRoot) is populated. Unless you are extending the schema, this groups should remain empty."
    $SchemaAdminsCollection.DistinguishedName
} else {
    Write-Output "The 'Schema Admins' group in $($DomainInformation.DNSRoot) is unpopulated."
}

### Admin SDHolder

In [23]:
Get-ADUser -Filter { AdminCount -eq 1 } -Server $DomainController |
    Format-Table -Property @('Enabled','SamAccountName','DistinguishedName')


## Computer Objects

### Default Computer Container

In [24]:
$DomainInformation.ComputersContainer

### Access Control List for the New Computers Container

In [25]:
Get-ADObject -Identity $DomainInformation.ComputersContainer -Server $DomainController -Property 'nTSecurityDescriptor' |
    Select-Object -ExpandProperty 'nTSecurityDescriptor' |
    Select-Object -ExpandProperty 'Access' |
    Group-Object -Property 'ActiveDirectoryRights' |
    Sort-Object -Property 'Count' -Descending

### Count of all Compouter Objects in the Domain

In [26]:
$ComputerCollection = @(Get-ADComputer -Filter * -Server $DomainController -Properties @('operatingSystem'))
$ComputerCollection.Count.ToString('#,##0')

### Summary of Operating Systems in the Domain

In [27]:
$ComputerCollection |
    Group-Object -Property 'OperatingSystem' |
    Select-Object -Property @(
        @{Name = 'Total';Expression={$_.Count}},
        @{Name = 'OperatingSystem';Expression={$_.Name}}
    ) |
    Sort-Object -Property 'Total' -Descending

### Table of All Server Based Computer Objects

In [28]:
Get-ADComputer -Filter { operatingSystem -like '*Server*' } -Properties @('operatingSystem','IPv4Address') |
Format-Table -Property @('Name','DNSHostName','IPv4Address','OperatingSystem')

## Group Objects

### Group Information

In [29]:
Get-ADGroup -Identity 'Domain Admins' -Server $DomainController

### Membership of a Group

In [30]:
$MemberCollection = Get-ADGroupMember -Identity 'Administrators' -Server $DomainController
$MemberCollection |
    Format-Table -Property @('SamAccountName','ObjectClass','DistinguishedName')

## Organizational Units

### Count of Organizational Units

In [31]:
@(Get-ADOrganizationalUnit -Filter * -Server $DomainController).Count.ToString('#,##0')

## User Objects

### Default User Container

In [32]:
$DomainInformation.UsersContainer

### Access Control List for the New Users Container

In [33]:
Get-ADObject -Identity $DomainInformation.UsersContainer -Server $DomainController -Property 'nTSecurityDescriptor' |
    Select-Object -ExpandProperty 'nTSecurityDescriptor' |
    Select-Object -ExpandProperty 'Access' |
    Sort-Object -Property @('IdentityReference','ActiveDirectoryRights') -Unique |
    Format-Table @('AccessControlType','IdentityReference','IsInherited','ActiveDirectoryRights')

### Count of all Disabled Accounts

In [34]:
@(Get-ADUser -Filter { Enabled -eq $false }).Count.ToString('#,##0')

### Users with Expired Password

In [35]:
$UsersWithExpiredPasswordCollection = @(
    Search-ADAccount -PasswordExpired -Server $DomainController -UsersOnly |
    Select-Object -ExpandProperty 'DistinguishedName'
    )
$UsersWithExpiredPasswordCollection

### Account that have Passwords that Never Expire

In [36]:
$UserswithPasswordsThatNeverExpireCollection = Search-ADAccount -PasswordNeverExpires -Server $DomainController -UsersOnly
$UserswithPasswordsThatNeverExpireCollection |
    Get-ADUser -Server $DomainController -Properties @('LastLogonDate','PasswordLastSet') |
    Format-Table -Property @('Enabled','SamAccountName','LastLogonDate','PasswordLastSet','DistinguishedName')

### Locked out Accounts

In [37]:
$LockedOutUserCollection = Search-ADAccount -LockedOut -Server $DomainController -UsersOnly
$LockedOutUserCollection |
    Get-ADUser -Server $DomainController -Properties @('LastLogonDate','LockedOut','BadLogonCount','LastBadPasswordAttempt') |
    Format-Table -Property @('Enabled','SamAccountName','LockedOut','BadLogonCount','LastLogonDate','LastBadPasswordAttempt','DistinguishedName')

### Expired User Objects

In [38]:
$Today = Get-Date
$PropertyList = @('Enabled', 'AccountExpirationDate', 'GivenName', 'SurName', 'Name', 'SamAccountName', 'UserPrincipalName', 'EmailAddress', 'Title', 'Department', 'Company', 'EmployeeID', 'EmployeeNumber', 'DistinguishedName')
[System.Array]$ExpiredUserCollection = Get-ADUser -Filter { AccountExpirationDate -lt $Today } -Properties $PropertyList -Server $DomainController
$ExpiredUserCollection

## Group Managed Service Accounts

### Display any KDS Root Keys Present

In [39]:
Get-ADObject -Filter * -SearchBase "CN=Master Root Keys,CN=Group Key Distribution Service,CN=Services,CN=Configuration,$($DomainInformation.DistinguishedName)"

### All Active Directory Group Managed Service Accounts

In [40]:
$gSMACollection = Get-ADServiceAccount -Filter * -Server $DomainController
if($gSMACollection) {
    $gSMACollection |
     Format-Table -Property @(
         'Enabled',
         'Name',
         'SamAccountName',
         'ObjectClass'
         'DistinguishedName'
     )
} else {
    Write-Output "No Group Managed Service Accounts are present in $($DomainInformation.DNSRoot)"
}

## All Active Directory Get CmdLets

In [41]:
Get-Command -Module 'ActiveDirectory' |
    Where-Object { $_.Name -match '^Get-*' }

In [42]:
Write-Output "[i] Run Time: $([System.Math]::Floor($RunTime.Elapsed.TotalHours).ToString('#,#00')):$($RunTime.Elapsed.Minutes.ToString('00')):$($RunTime.Elapsed.Seconds.ToString('00'))"

-- Robert M. Toups, Jr.